In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math

In [7]:
def socialdist(dist,thresh):
    viol=[]
    for pt in dist:
        vio=[]
        for j in dist:
            if j!=pt:
                vio.append(((pt[0]-j[0])**2+(pt[1]-j[1])**2)**(0.5)<thresh)
        viol.append(np.array(vio[-1]).all())       
    return viol   

In [10]:
cap=cv2.VideoCapture(r"C:\Users\ARYAN JAIN\Desktop\people.mp4")
backg=cv2.bgsegm.createBackgroundSubtractorMOG()
ret,frame1=cap.read()
ret,frame2=cap.read()
while(1):
    _,frame1=cap.read()
    #frame=cv2.absdiff(frame1,frame2)
    bgmask=backg.apply(frame1)
    bgg=cv2.bitwise_not(bgmask)
    res=cv2.bitwise_and(frame1,frame1,mask=bgg)
    res1=cv2.bitwise_and(frame1,frame1,mask=bgmask)
    res2=cv2.addWeighted(res,1,res1,1,0)
    _,contours,_=cv2.findContours(bgmask, cv2.CHAIN_APPROX_SIMPLE,cv2.RETR_TREE)
    #cv2.drawContours(res2,contours,-1,(255,0,0),thickness=2)
    dist=[]
    point=[]
    for contour in contours:
        (x,y,w,h)=cv2.boundingRect(contour)
        if cv2.contourArea(contour)>400:
            point.append((x,y,w,h))
            dist.append((2*x+w/2,2*y+h/2))
    violators=socialdist(dist,700)
    for i in range(len(violators)):
        if(violators[i]):
            cv2.rectangle(res2,(point[i][0],point[i][1]),(point[i][0]+point[i][2],point[i][1]+point[i][3]),(0,0,255),2)
        else:
            cv2.rectangle(res2,(point[i][0],point[i][1]),(point[i][0]+point[i][2],point[i][1]+point[i][3]),(0,255,0),2)
    cv2.imshow("f1",res)
    cv2.imshow("f2",frame1)
    cv2.imshow("res",res2)
    #frame1=frame2
    #ret,frame2=cap.read()
    if cv2.waitKey(1)==ord('q') & 0xff:
        break

cap.release()
cv2.destroyAllWindows()